In [95]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [96]:
import zipfile
dataset = zipfile.ZipFile('Mangga.zip', 'r')
dataset.extractall()
dataset.close()

In [113]:
import os
os.listdir('Mangga')

['Mangga Alpukat', 'Mangga Harum Manis', 'Mangga Gedong Gincu']

In [114]:
# Direct
dataset_dir = 'Mangga'

In [115]:
data = []
labels = []

In [116]:

# Preprocessing: Membaca gambar dan menyimpan data serta labelnya
for class_dir in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_dir)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            # Membaca gambar
            img = cv2.imread(image_path)
            img = cv2.resize(img, (299, 299))  # Resize sesuai InceptionV3
            data.append(img)
            labels.append(class_dir)

In [117]:
# Konversi ke numpy array
data = np.array(data)
labels = np.array(labels)


In [118]:
# One-hot encode label
label_encoder = {label: i for i, label in enumerate(np.unique(labels))}
labels = np.array([label_encoder[label] for label in labels])

In [119]:
# Split data menjadi train dan test (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels)



In [120]:

# Augmentasi menggunakan ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [121]:

# Siapkan generator data training
train_datagen = datagen.flow(X_train, y_train, batch_size=32)

# Siapkan model InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Bangun model klasifikasi
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(len(np.unique(labels)), activation='softmax')  # Kelas sesuai jumlah label unik
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [122]:
# Siapkan model InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Bangun model klasifikasi
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(len(np.unique(labels)), activation='softmax')  # Kelas sesuai jumlah label unik
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [123]:
# Latih model
history = model.fit(
    train_datagen,
    steps_per_epoch=len(X_train) // 64,
    epochs=10,
    validation_data=(X_test / 255.0, y_test)  # Normalisasi data test
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 128s 42s/step - accuracy: 0.4032 - loss: 1.1002 - val_accuracy: 1.0000 - val_loss: 0.0655
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 70s 35s/step - accuracy: 1.0000 - loss: 0.0147 - val_accuracy: 0.6667 - val_loss: 0.7902
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 68s 37s/step - accuracy: 1.0000 - loss: 6.4582e-04 - val_accuracy: 0.3333 - val_loss: 5.7708
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 86s 37s/step - accuracy: 1.0000 - loss: 2.9330e-05 - val_accuracy: 0.3333 - val_loss: 8.9015
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 70s 33s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.3333 - val_loss: 7.5218
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 83s 35s/step - accuracy: 1.0000 - loss: 2.5992e-05 - val_accuracy: 0.3333 - val_loss: 5.3823
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 68s 37s/step - accuracy: 0.9889 - loss: 0.0686 - val_accuracy: 0.3333 - val_loss: 6.0164
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 83s 35s/step - accuracy: 0.9667 - loss: 0.1353 - val_accuracy: 0.3333 - val_loss: 20.52

In [125]:

# Evaluasi model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3333 - loss: 843.6144
Test Accuracy: 33.33%
